In [10]:
import pandas as pd   
import pandas_gbq as pgbq 
import os
import re
import sys

In [11]:
path_note = os.getcwd()
path_src = re.sub(r'(.)notebooks','',path_note)
sys.path.append(os.path.abspath(f'{path_src}/'))

In [12]:
import google.auth

user = os.getlogin()
#linux
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/home/andre/.ssh/my-chave-gcp-devsamelo2.json'
#windows
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r'C:\Users\andre\.ssh\my-chave-gcp-devsamelo2.json'
#[Environment]::SetEnvironmentVariable("GOOGLE_APPLICATION_CREDENTIALS","C:\Users\andre\.ssh\my-chave-gcp-devsamelo2.json","User")

#credencial = service_account.Credentials.from_service_account_file(key)
credencial, pid = google.auth.default(     
    scopes=[
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/bigquery",
    "https://www.googleapis.com/auth/cloud-platform"
])

In [18]:

strsql = '''
SELECT custo
, tipo_custo
, 'debito'
, dt_mes_base
, valor_custo
, dt_custo
 FROM devsamelo2.dev_domestico.custo_2023_excel --where dt_mes_base = '2023-12-01' --where tipo_custo is null --= 'compra'
#  union all
#  SELECT custo_credito  de_para
# , tipo_custo_credito tipo_custo
# , 'credito' source
# , dt_mes_base
#  FROM devsamelo2.dev_domestico.credito_2023_excel --where tipo_custo_credito is null

'''

#--regexp_extract(custo, r'^PIX TRANSF .(\w.+)') is not null
#--regexp_extract(custo, r'^(SABESP)') = 'SABESP'

df = pgbq.read_gbq(strsql, project_id='devsamelo2' , credentials=credencial, progress_bar_type=None)

In [19]:
print('df',len(df))
print('nulo',len(df[(df["tipo_custo"].isnull())]))
print('not nulo',len(df[(df["tipo_custo"].notnull())]))

df 1005
nulo 0
not nulo 1005


In [15]:
dfnull=df[(df["tipo_custo"].isnull())]

In [17]:
dfnull
dfnull.to_csv('sql/depara.csv')

In [9]:
dfnull = dfnull.rename(columns={'valor': 'tipo_custo'})
dfnull = dfnull.rename(columns={'custo': 'descricao'})

from mytypedestinmoney_v1 import my_destin_pushout_csv
dfnull = my_destin_pushout_csv(dfnull, r'C:\Users\andre\Documents\github\my-money-family\admin\src\csv\08-11-2023-type.csv')


# import re
# dfnull["valor"] = dfnull.apply(lambda x: ' '.join(re.findall(r'^ELECTRON+(\w+)',x["de_para"])) if x["de_para"].find('ELECTRON')>=0 else
#                                                     (' '.join(re.findall(r'^PIX+\b.+QRS+(\b.+[a-zA-Z])',x["de_para"])) if x["de_para"].find('PIX QRS')>=0 else
#                                                      (' '.join(re.findall(r'^PIX+\b.+TRANSF+(\b.+[a-zA-Z]+)',x["de_para"])) if x["de_para"].find('PIX TRANSF')>=0 else
#                                                         ' '.join(re.findall(r'^(\w+)',x["de_para"])))
#                                                                  )
#                                                         ,axis=1)

Etapa INTERMEDIARIA DO PUSHOUT para classificação do tipo de custo 

Total tipos_encontrados:6 / tipos_nao_encontrados:0 / tipo_generico:0



,descricao,tipo_custo,f0_,dt_mes_base,valor_custo,dt_custo
3,ELECTRONMARY PAPER,loja,debito,2023-12-01,13.0,2023-12-04
4,ELECTRONPAG*JoseSev,pessoa,debito,2023-12-01,10.0,2023-12-04
5,ELECTRONPAG*ArianaD,pessoa,debito,2023-12-01,4.0,2023-12-04
6,ELECTRONPALMAS DO T,padaria,debito,2023-12-01,13.0,2023-12-04
7,ELECTRONRAMOV TRIMA,alimentacao,debito,2023-12-01,16.0,2023-12-04
8,ELECTRONIMW VILA ME,igreja,debito,2023-12-01,36.0,2023-12-04


In [92]:
dfnull

,de_para,valor,f0_,dt_mes_base
59,001,001,credito,2023-05-01
210,001,001,credito,2023-07-01
356,Fgr Comercio-ct 01/03,Fgr,credito,2023-09-01
361,Google Cloud 98nxrw,Google,credito,2023-09-01
452,M Jarouche Comercio(03/03),M,credito,2023-10-01
454,Artemis Comercio De(03/03),Artemis,credito,2023-10-01
455,Mp *klmcentr-ct O (03/04),Mp,credito,2023-10-01
456,Aluguel (03/06),Aluguel,credito,2023-10-01
461,Fgr Comercio-ct (02/03),Fgr,credito,2023-10-01
679,Pag*geraldot-ct Atova,Pag,credito,2023-03-01


In [68]:
dfnonull=df[(df["valor"].notnull())]

In [71]:
df_new = pd.concat([dfnonull, dfnull])

In [72]:
#escrever novos info no csv existente - mode='a'

import csv
df_new.to_csv(f'{path_src}/csv/type_ia_v2.csv',sep=';', encoding='utf-8', header=True, index=False,    quoting=csv.QUOTE_NONNUMERIC,
    escapechar="\\",
    doublequote=False, mode='w')

In [75]:
df = pd.read_csv(f'{path_src}/csv/type_ia_v2.csv', sep=';', usecols=['de_para','valor'])#window incluir ecoding
# # encoding='latin-1'
# # configurar ';', e incluir aspas duplas '"', e tipo iso-8859-1
# df_type = df_type[(df_type["valor"].notnull())].drop_duplicates()#.unique() | .notnull() | .isnull()


In [76]:
print('df',len(df))
print('nulo',len(df[(df["valor"].isnull())]))
print('not nulo',len(df[(df["valor"].notnull())]))

df 1444
nulo 0
not nulo 1444


In [ ]:
# df_type["valor"] = df_type.apply(lambda x: "debito_comercio" if x["de_para"].find('ELECTRON')>=0 else 
#                                 ( "Shopee" if x["de_para"].find('PIX QRS Shopee')>=0 else 
#                                 ( "pix" if x["de_para"].find('PIX')>=0 else x["de_para"]))
#                                 ,axis=1)
# df_type=df_type[df_type['valor'].notnull()].copy()

In [ ]:
# import re
# df_type["valor"] = df_type.apply(lambda x: ' '.join(re.findall(r'^Pag+\*(\w+)',x["de_para"])) if x["de_para"].find('Pag*')>=0 else 
#                                                         ' '.join(re.findall(r'^(\w+)',x["de_para"])
#                                                                  )
#                                                         ,axis=1)

In [ ]:
# #escrever novo csv - mode='w'
# import csv
# df_type.to_csv(f'{path_src}/csv/type_ia_v2.csv',sep=';', encoding='utf-8', header=True, index=False,    quoting=csv.QUOTE_NONNUMERIC,
#     escapechar="\\",
#     doublequote=False, mode='a')

In [157]:
#df_type_1aver = df_type[df_type['valor']!="compras"].copy()

In [158]:
#escrever novo csv - mode='w'
# import csv
# df_type_1aver.to_csv(f'{path_src}/csv/type_ia_v2.csv',sep=';', encoding='utf-8', header=True, index=False,    quoting=csv.QUOTE_NONNUMERIC,
#     escapechar="\\",
#     doublequote=False, mode='w')

In [159]:
# df_type_2aver=df_type[df_type['valor']=="compras"].copy()
# if len(df_type_2aver)>0:
#     df_type_2aver["valor"] = df_type_2aver.apply(lambda x: "debito_comercio" if x["de_para"].find('ELECTRON')>=0 else 
#                                     ("Shopee" if x["de_para"].find('PIX QRS Shopee')>=0 else 
#                                     ( "pix" if x["de_para"].find('PIX')>=0 else None))
#                                     ,axis=1)
#     df_type_2aver=df_type_2aver[df_type_2aver['valor'].notnull()].copy()


ValueError: Columns must be same length as key